In [1]:
import pandas as pd

In [2]:
details_df = pd.read_csv('../Data/Seleccion/details.csv')

In [3]:
details_df = details_df.set_index('EVENT_ID')

In [4]:
#Comprobamos que atributos tienen valores nulos
details_df.isnull().any()

EPISODE_ID            True
EVENT_TYPE           False
BEGIN_DATE_TIME      False
END_DATE_TIME        False
INJURIES_DIRECT      False
INJURIES_INDIRECT    False
DEATHS_DIRECT        False
DEATHS_INDIRECT      False
DAMAGE_PROPERTY       True
DAMAGE_CROPS          True
MAGNITUDE             True
BEGIN_LAT             True
BEGIN_LON             True
END_LAT               True
END_LON               True
dtype: bool

In [5]:
#Eliminamos aquellos eventos que no tengan asociado un episodio (tormenta)
details_df = details_df[details_df.EPISODE_ID.notnull()]

In [6]:
#Convertimos los episodios a tipo entero
details_df.loc[:, 'EPISODE_ID'] = details_df['EPISODE_ID'].apply(lambda x: int(x))

In [7]:
#Damos formato a las fechas
details_df.loc[:, 'BEGIN_DATE_TIME'] = pd.to_datetime(details_df['BEGIN_DATE_TIME'])
details_df.loc[:, 'END_DATE_TIME'] = pd.to_datetime(details_df['END_DATE_TIME'])

In [8]:
#Damos el valor por defecto de 0 al numero de muertes directas e indirectas, perdidas economicas generales y en cultivos,
# heridos directos e indirectos
details_df[['INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DAMAGE_CROPS',
            'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS']] = details_df[['INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DAMAGE_CROPS',
            'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS']].fillna('0')

In [9]:
#Convertismos las perdidas de propiedades y de cultivos en un numero entero transformando las distintas magnitudes
# en una comun ($)
def procesar_perdidas(x):
    if x == '0' or x == 0:
        return int(x)
    else:
        if x[-1] == 'K':
            mult = 1e3
        elif x[-1] == 'M':
            mult = 1e6
        else:
            mult = 1
        
        return int(float(x[:-1]) * mult)

In [10]:
details_df['DAMAGE_PROPERTY'] = details_df['DAMAGE_PROPERTY'].apply(procesar_perdidas)

In [11]:
details_df['DAMAGE_CROPS'] = details_df['DAMAGE_CROPS'].apply(procesar_perdidas)

In [12]:
# Observamos que los valores de las perdidas en muchos casos es la misma, lo cual indica que se introdujeron
# de forma poco precisa
details_df['DAMAGE_PROPERTY'].unique()

array([        0,      1000,    125000,    250000,     10000,      5000,
          150000,    200000,     75000,     50000,      2000,      3000,
           15000,     68000,      2500,       100,      8000,     40000,
           70000,     13000,    350000,      6000,     20000,      1500,
            2600,     60000,      7500,    120000,     45000,     12500,
          175000,   2000000,   1000000,    750000,    100000,     90000,
        12000000,     30000,      9000,    500000,    260000,   1250000,
            7000,      4000,     12000,    160000,    400000,     21000,
           25000,     28000,     55000,     80000,       250,       600,
           35000,   3000000,     11000,    226000,       500,  23000000,
          300000,  29000000,      9500,   8000000,  20000000,  80000000,
        50000000, 100000000,  10000000,  45000000, 150000000,  30000000,
        90000000,  15000000,   5000000,       800,     53000,     85000,
         4000000,   2900000,   6000000,   2700000, 

In [13]:
# Lo mismo ocurre con las perdidas economicas en cultibos
details_df.DAMAGE_CROPS.unique()

array([        0,      5000,     50000,     10000,   1000000,    250000,
           25000,    100000,      2000,    200000,      1000,     20000,
           15000,  10000000,    500000,    300000,     30000,  60000000,
        30000000, 100000000,  45000000, 150000000,  50000000,  90000000,
        15000000,   5000000,      3000,   2000000,   4000000,      7000,
            1500,       500,     75000,    400000,      8000,       750,
         3030000,    620000,   8000000,   7000000,   3000000,   6000000,
             250,       200,       100,      2500,     12000,      4000,
           40000,      6000,       300,  70000000])

In [14]:
# Quitamos los nulos de las magnitudes y los ponemos a 0
details_df['MAGNITUDE'] = details_df['MAGNITUDE'].fillna(0)

In [27]:
# Pasamos a preprocesar las posiciones iniciales y finales
import matplotlib.pyplot as plt

plt.scatter(details_df.BEGIN_LAT, details_df.BEGIN_LON)
plt.show()

plt.scatter(details_df.END_LAT, details_df.END_LON)
plt.show()

#Observamos que estan correctamente introducidos los que hay

In [16]:
#Hay demasiados nulos en todas las posiciones
details_df['END_LAT'].isnull().sum()

6964

In [17]:
"""
Medidas a tomar:

1. Establecer como valores INI_LON y INI_LAT los de locations.
2. Establecer como valores protipicos de INI_LON e INI_LAT:
        if hay algun evento en el mismo episodio con estos valores: la media de ellos
        else 
            if ese evento tiene END_LON y END_LAT añadirlo
            elif algun evento de ese episodio tiene END_LON y END_LAT: la media de ellos
            else 
                eliminarlo

3. Establecer como valores END_LON y END_LAT los de details.csv
4. Establecer como valores protipicos de END_LON y END_LAT:
       if hay algun evento con el mismo episodio: la media de ellos
       else
           if ese evento tiene INI_LON y INI_LAT añadirlo
           elif algun evento tiene INI_LON y INI_LAT del mismo episodio: media de ellos
           else
               eliminarlo
"""

'\nMedidas a tomar:\n\n1. Establecer como valores INI_LON y INI_LAT los de locations.\n2. Establecer como valores protipicos de INI_LON e INI_LAT:\n        if hay algun evento en el mismo episodio con estos valores: la media de ellos\n        else \n            if ese evento tiene END_LON y END_LAT a\xc3\xb1adirlo\n            elif algun evento de ese episodio tiene END_LON y END_LAT: la media de ellos\n            else \n                eliminarlo\n\n3. Establecer como valores END_LON y END_LAT los de details.csv\n4. Establecer como valores protipicos de END_LON y END_LAT:\n       if hay algun evento con el mismo episodio: la media de ellos\n       else\n           if ese evento tiene INI_LON y INI_LAT a\xc3\xb1adirlo\n           elif algun evento tiene INI_LON y INI_LAT del mismo episodio: media de ellos\n           else\n               eliminarlo\n'

In [18]:
# Tenemos 5259 eventos que han provocado perdidas economicas de caracter general
details_df[details_df.DAMAGE_PROPERTY > 0].shape[0]

5259

In [19]:
# Tenemos 493 eventos que han provocado perdidas economicas en cultivos
details_df[details_df.DAMAGE_CROPS > 0].shape[0]

493

In [20]:
# Tenemos 164 eventos que han provocado heridos directamente
details_df[details_df.INJURIES_DIRECT > 0].shape[0]

164

In [21]:
# Tenemos 46 eventos que han provocado heridos indirectamente
details_df[details_df.INJURIES_INDIRECT > 0].shape[0]

46

In [22]:
# Tenemos 91 eventos que han provocado muertes directamente
details_df[details_df.DEATHS_DIRECT > 0].shape[0]

91

In [23]:
# Tenemos 46 eventos que han provocado muertes indirectamente
details_df[details_df.INJURIES_INDIRECT > 0].shape[0]

46

In [24]:
details_df.to_csv('../Data/Preproceso/details.csv')

In [25]:
details_df['EVENT_TYPE'].unique()

array(['Thunderstorm Wind', 'Extreme Cold/Wind Chill', 'Dense Fog',
       'Strong Wind', 'Flood', 'Flash Flood', 'Ice Storm', 'Winter Storm',
       'High Wind', 'Winter Weather', 'Lake-Effect Snow', 'Hail',
       'Heavy Rain', 'Lightning', 'Heat', 'Funnel Cloud', 'Rip Current',
       'Frost/Freeze', 'Tornado', 'Cold/Wind Chill', 'Heavy Snow',
       'Excessive Heat', 'Blizzard', 'Drought', 'Wildfire', 'Dust Storm',
       'Freezing Fog', 'Coastal Flood', 'Dust Devil', 'Dense Smoke'], dtype=object)

In [26]:
details_df[details_df['EVENT_TYPE'] == 'Heat']

,EPISODE_ID,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,MAGNITUDE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
EVENT_ID,,,,,,,,,,,,,,,
125481,21217,Heat,2008-08-05 13:00:00,2008-08-05 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
125482,21217,Heat,2008-08-05 13:00:00,2008-08-05 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
125483,21217,Heat,2008-08-05 13:00:00,2008-08-05 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
125479,21217,Heat,2008-08-05 13:00:00,2008-08-05 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
116075,19617,Heat,2008-07-21 14:00:00,2008-07-21 17:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
589446,98050,Heat,2015-07-14 09:00:00,2015-07-14 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
589447,98050,Heat,2015-07-14 09:00:00,2015-07-14 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
589448,98050,Heat,2015-07-14 09:00:00,2015-07-14 16:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN
589388,98040,Heat,2015-07-13 12:00:00,2015-07-13 17:00:00,0,0,0,0,0,0,0.0,NaN,NaN,NaN,NaN


In [41]:
details_df[details_df['BEGIN_DATE_TIME'] > '2014-08-05'].shape

(2183, 15)